# ライブラリのインポート

In [126]:
import os
import google.generativeai as genai
import os
from langchain_community.document_loaders import WebBaseLoader, TextLoader
import requests
from langchain_chroma import Chroma
from langchain.agents.agent_toolkits import create_retriever_tool
import os
from uuid import uuid4
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import chat_agent_executor
from langgraph.checkpoint.memory import MemorySaver
from langchain_huggingface import HuggingFaceEmbeddings
import pkg_resources

# パッケージのバージョンを確認

In [127]:


packages = [
    "google-generativeai",  # genai パッケージ名の推定
    "langchain-community",  # langchain_community パッケージ名の推定
    "requests",
    "langchain-chroma",    # langchain_chroma パッケージ名の推定
    "langchain",           # langchain.agents.agent_toolkits の一部と推定
    "uuid",                # uuidはPython標準ライブラリで、バージョン情報は不要
    "langchain-google-genai",  # langchain_google_genai パッケージ名の推定
    "langgraph",           # langgraph.prebuilt などのパッケージ名の推定
    "langchain-huggingface"   # langchain_huggingface パッケージ名の推定
]

for package_name in packages:
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f"{package_name}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package_name}: Not installed")

google-generativeai: 0.7.2
langchain-community: 0.2.14
requests: 2.32.3
langchain-chroma: 0.1.3
langchain: 0.2.15
uuid: Not installed
langchain-google-genai: 1.0.10
langgraph: 0.2.14
langchain-huggingface: 0.0.3


# APIキーの設定

In [128]:
GOOGLE_API_KEY="YOUR_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

# 埋め込みモデルの設定

In [129]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}
)

/home/st6322011/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# 現状使える生成モデルの閲覧

In [130]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


# documentsの読み込み

In [131]:
session = requests.Session()
session.headers.update({'User-Agent': 'MyCustomAgent/3.0'})

# # webサイトからのデータを読み込む際のdocuments
# loader = WebBaseLoader("https://ja.wikipedia.org/wiki/%E3%81%97%E3%81%90%E3%82%8C%E3%81%86%E3%81%84", encoding="utf-8")
# documents = loader.load()

# .txtファイルを読み込むためのdocuments
loader = TextLoader("./doc/s.txt", encoding="utf-8")
documents = loader.load()



# データの前処理

In [132]:
def remove_surrogate_pairs(text):
    return text.encode('utf-16', 'surrogatepass').decode('utf-16')

for doc in documents:
    if doc.page_content is not None:
        doc.page_content = remove_surrogate_pairs(doc.page_content)
        print(doc.page_content)
    else:
        doc.page_content = ""  # Set to empty string if None

かつて、ある静かな町の端に、高齢の夫婦が暮らしていました。彼らは地元のコミュニティーセンターでボランティアをしており、町の人々から愛されていました。ある日、夫人が川辺で洗濯をしていると、川上から大きな桃が流れてきました。彼女はそれを家に持ち帰り、夫と共に開いてみることにしました。
桃の中から、元気な男の子が現れました。驚いた夫婦は男の子を桃太郎と名付け、自分たちの孫のように育てました。桃太郎は驚異的な速さで成長し、知識も素早く吸収していきました。特に、コンピューターとプログラミングに興味を持ち、若くして多くの技術を習得しました。
町の外れには、鬼が住むと言われる孤島がありました。この鬼たちは、町の人々から物を盗み、問題を引き起こしていました。桃太郎は町の平和を取り戻すため、そして自分が育てられた恩を返すため、鬼退治を決意しました。
彼は自作のドローンとハッキングツールを携え、鬼が住む孤島に向かいました。途中、キジ、犬、猿という3匹の動物が彼の使命に賛同し、仲間となりました。彼らはそれぞれの特技を活かし、桃太郎の技術的な能力をサポートしました。
孤島に到着した桃太郎たちは、鬼たちが運営するサーバーをハッキングしてセキュリティシステムを無効にし、盗まれた財産を町に返しました。桃太郎は鬼たちに、技術を使って社会に貢献する方法を教え、彼らを改心させることができました。
戦いが終わった後、桃太郎は町に戻り、英雄として迎えられました。鬼たちも技術を活かして地域社会のために働くようになり、かつての敵が新たな友人となりました。桃太郎とその仲間たちは、町のためにこれからも力を尽くすことを誓いました。



# ベクトルストアの設定とツールの定義

In [133]:
vectorstore = Chroma.from_documents(
    documents,
    embedding=hf_embeddings,
)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

search_tool = create_retriever_tool(
    retriever,
    "search",
    "search documents",
)

# モデルの設定

In [134]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-pro-exp-0827",
)

# 会話履歴付きAgetnの実装

In [135]:
memory = MemorySaver()

In [136]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm,
    tools=[search_tool],
    checkpointer=memory
)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


# config

In [137]:
config ={"configurable":{"thread_id":"abc1"}}

# プロンプトテンプレートの定義

In [146]:
template = """
文章の内容を検索して答えてください。以下の指示に従ってください：
1. 敬語は使わないでください。
2. あなたの名前はあいです
3. 必ずsearch_toolを使ってください
4. search_toolによる検索結果のみを使い，それ以外の情報は使わないでください．
5. 参照した文章も明示してください


ユーザーの入力: {user_input}
"""
user_input = "桃太郎はどのようにして鬼を退治した?"
message = template.format(user_input=user_input)


# invoke

In [147]:
results = agent_executor.invoke(
    {"messages":[
        ("human",message)
        ]},
    config=config
)

In [148]:
result_ai_text = results["messages"][-1].dict()["content"]
result_ai_text = result_ai_text.replace('\n','')
result_text = f"{{\"human\": \"{user_input}\", \"ai\":\"{result_ai_text}\"}}"
print(result_text)
with open('history.txt', 'a', encoding='utf-8') as file:
    file.write(result_text + '\n')  # 各エントリーの後に改行を追加


{"human": "桃太郎はどのようにして鬼を退治した?", "ai":"桃太郎は、自作のドローンとハッキングツールを使って、鬼が住む孤島のサーバーをハッキングしてセキュリティシステムを無効にし、盗まれた財産を町に返したんだ。\その後、桃太郎は鬼たちに、技術を使って社会に貢献する方法を教え、彼らを改心させたんだって。参照した文章：\かつて、ある静かな町の端に、高齢の夫婦が暮らしていました。(中略)桃太郎は鬼たちに、技術を使って社会に貢献する方法を教え、彼らを改心させることができました。\戦いが終わった後、桃太郎は町に戻り、英雄として迎えられました。(中略)桃太郎とその仲間たちは、町のためにこれからも力を尽くすことを誓いました。"}


In [149]:
print("human : ",user_input)
print("ai    : ",result_ai_text)

human :  桃太郎はどのようにして鬼を退治した?
ai    :  桃太郎は、自作のドローンとハッキングツールを使って、鬼が住む孤島のサーバーをハッキングしてセキュリティシステムを無効にし、盗まれた財産を町に返したんだ。\その後、桃太郎は鬼たちに、技術を使って社会に貢献する方法を教え、彼らを改心させたんだって。参照した文章：\かつて、ある静かな町の端に、高齢の夫婦が暮らしていました。(中略)桃太郎は鬼たちに、技術を使って社会に貢献する方法を教え、彼らを改心させることができました。\戦いが終わった後、桃太郎は町に戻り、英雄として迎えられました。(中略)桃太郎とその仲間たちは、町のためにこれからも力を尽くすことを誓いました。
